# DNA metilazione, beta values e ambiente

Obiettivi del notebook:
- Capire cosa sono i **beta values** (valori di metilazione tra 0 e 1).
- Esplorare la distribuzione della metilazione in un dataset sintetico.
- Collegare metilazione e variabile di esposizione ambientale (es. PM₂.₅).
- Eseguire un'analisi molto semplice di differenza di metilazione tra gruppi.

_Tutti i dati sono simulati per fini didattici._

In [ ]:
from __future__ import annotations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline

np.random.seed(42)

## Simulazione di un piccolo dataset di metilazione

Creiamo:
- 200 CpG (righe)
- 40 campioni (20 controlli, 20 esposti)
- per la maggior parte delle CpG, stessa distribuzione nei due gruppi
- per un piccolo sottoinsieme (es. 10 CpG), le medie di metilazione sono diverse tra i gruppi.

In [ ]:
n_cpg = 200
n_ctrl = 20
n_exp = 20
n_samples = n_ctrl + n_exp

cpg_ids = [f"cg{str(i).zfill(6)}" for i in range(1, n_cpg + 1)]
sample_ids = [f"CTRL_{i+1}" for i in range(n_ctrl)] + [f"EXP_{i+1}" for i in range(n_exp)]

# Gruppi: 0 = controllo, 1 = esposto
group = np.array([0]*n_ctrl + [1]*n_exp)
group_series = pd.Series(group, index=sample_ids, name="group")

# Variabile di esposizione continua (es. PM2.5), più alta nel gruppo esposto
pm25_base_ctrl = 15
pm25_base_exp = 30
pm25 = np.where(group == 0,
                np.random.normal(pm25_base_ctrl, 3, size=n_samples),
                np.random.normal(pm25_base_exp, 3, size=n_samples))
pm25_series = pd.Series(pm25, index=sample_ids, name="PM25")

# Metilazione di base: beta ~ Beta(a, b)
beta_matrix = np.random.beta(a=2, b=5, size=(n_cpg, n_samples))

# Introduciamo differenze per 10 CpG "sensibili" all'esposizione
diff_cpg_idx = np.arange(0, 10)  # le prime 10 CpG
for idx in diff_cpg_idx:
    # aumento di metilazione nei soggetti esposti
    beta_matrix[idx, group == 1] += 0.2

# Limitiamo i valori a [0, 1]
beta_matrix = np.clip(beta_matrix, 0, 1)

beta_df = pd.DataFrame(beta_matrix, index=cpg_ids, columns=sample_ids)
beta_df.head()

## Esplorare la distribuzione globale dei beta values

Guardiamo la distribuzione complessiva dei beta values (tutti CpG, tutti campioni).

In [ ]:
all_beta = beta_df.values.ravel()
plt.hist(all_beta, bins=20)
plt.xlabel("Beta (metilazione)")
plt.ylabel("Conteggio")
plt.title("Distribuzione globale dei beta values (tutti CpG, tutti campioni)")
plt.show()

## Confronto di un CpG tra controlli ed esposti

Scegliamo una CpG sensibile all'esposizione (una delle prime 10) e confrontiamo la distribuzione tra gruppi.

In [ ]:
cpg_example = cpg_ids[0]  # una CpG con effetto
values_ctrl = beta_df.loc[cpg_example, group_series == 0]
values_exp = beta_df.loc[cpg_example, group_series == 1]

print("CpG di esempio:", cpg_example)
print("Media CTRL:", round(values_ctrl.mean(), 3))
print("Media EXP :", round(values_exp.mean(), 3))

plt.hist(values_ctrl, alpha=0.7, bins=10, label="CTRL")
plt.hist(values_exp, alpha=0.7, bins=10, label="EXP")
plt.xlabel("Beta (metilazione)")
plt.ylabel("Conteggio")
plt.title(f"Distribuzione beta per {cpg_example}")
plt.legend()
plt.show()

## Metilazione vs esposizione continua (PM₂.₅)

Per la stessa CpG, mettiamo in relazione il beta value con la concentrazione di PM₂.₅.

In [ ]:
cpg_values = beta_df.loc[cpg_example, :]
plt.scatter(pm25_series, cpg_values)
plt.xlabel("PM2.5 (µg/m³)")
plt.ylabel("Beta metilazione")
plt.title(f"Metilazione di {cpg_example} vs PM2.5")
plt.show()

r, pval = stats.pearsonr(pm25_series.values, cpg_values.values)
print(f"Correlazione Pearson r = {r:.3f}, p-value = {pval:.3e}")

## Analisi semplice di differenza di metilazione (DMC)

Eseguiamo, per ogni CpG:
- la differenza di media (EXP - CTRL)
- un t-test di Student tra gruppi

_Nota_: è un esempio didattico; nelle analisi reali bisogna usare modelli più robusti, correzioni multiple, ecc.

In [ ]:
results = []
for cpg in cpg_ids:
    vals = beta_df.loc[cpg, :]
    vals_ctrl = vals[group_series == 0]
    vals_exp = vals[group_series == 1]
    diff = vals_exp.mean() - vals_ctrl.mean()
    tstat, pval = stats.ttest_ind(vals_exp, vals_ctrl, equal_var=False)
    results.append({
        "CpG": cpg,
        "mean_ctrl": vals_ctrl.mean(),
        "mean_exp": vals_exp.mean(),
        "diff_exp_minus_ctrl": diff,
        "pval": pval,
    })

res_df = pd.DataFrame(results).set_index("CpG")
res_df_sorted = res_df.sort_values("pval")
res_df_sorted.head(10)

## Selezionare CpG potenzialmente differenzialmente metilate

Applichiamo una soglia molto semplice:
- |diff| ≥ 0.1
- p-value ≤ 0.01

_Solo per scopi illustrativi._

In [ ]:
threshold_diff = 0.1
threshold_p = 0.01

hits = res_df[(res_df["diff_exp_minus_ctrl"].abs() >= threshold_diff) & (res_df["pval"] <= threshold_p)]
print(f"Numero di CpG che passano le soglie: {len(hits)}")
hits.sort_values("pval").head(10)

Questa è un'analisi estremamente semplificata, ma illustra il concetto di **DMC** (Differentially Methylated CpG) in un contesto esposizione vs controllo.

In studi reali di epigenomica ambientale si lavora spesso con:
- centinaia di migliaia di CpG (array 450k/EPIC)
- modelli lineari (es. limma) con covariate (età, sesso, cell composition, batch)
- correzione per test multipli (FDR).

Qui però abbiamo visto i mattoncini di base in Python.